In [225]:
import numpy as np
import scipy
from scipy.special import expit

In [333]:
class BaseSmoothOracle:
    """
    Базовый класс для реализации оракулов.
    """
    def func(self, w):
        """
        Вычислить значение функции в точке w.
        """
        return lambda X, y: expit(y * X.dot(w))

    def grad(self, w):
        """
        Вычислить значение градиента функции в точке w.
        """
        return lambda X, y: expit(y * X.dot(w))


In [610]:
class BinaryLogistic(BaseSmoothOracle):
    """
    Оракул для задачи двухклассовой логистической регрессии.
    
    Оракул должен поддерживать l2 регуляризацию.
    """
    
    def __init__(self, l2_coef):
        """
        Задание параметров оракула.
        
        l2_coef - коэффициент l2 регуляризации
        """
        self.l2_coef = l2_coef
     
    def func(self, X, y, w):
        """
        Вычислить значение функционала в точке w на выборке X с ответами y.
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
        
        w - одномерный numpy array
        """
        act_func = super().func(w)
        
        return - np.log(act_func(X, y)).mean() + \
                 self.l2_coef*(w.dot(w)) / 2

        
    def grad(self, X, y, w):
        """
        Вычислить градиент функционала в точке w на выборке X с ответами y.
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
        
        w - одномерный numpy array
        """
        grad_func = super().grad(w)
        
        return (-(X.transpose()).multiply((1 - grad_func(X, y)))).dot(y) / y.shape[0] + self.l2_coef * w
        